# Notebook Semana-3
## Extração de Features, Bordas e Cantos de uma Imagem
O objetivo deste notebook é explorar técnicas de extração de features (características) de uma imagem, incluindo métodos para detectar bordas e cantos. A extração de features é uma etapa fundamental no processamento de imagens, pois permite identificar pontos de interesse que são úteis para análises e aplicações em visão computacional.

Neste notebook, abordaremos:
- Filtros de Sobel: Para calcular gradientes e detectar mudanças de intensidade na imagem.
- Operador de gradiente: Para detecção de bordas usando a primeira derivada.
- Detector de Bordas de Canny: Um método popular para detecção precisa de bordas.
- Detector de Cantos de Harris e Shi-Tomasi: Para identificar pontos de interesse (cantos) na imagem.

In [ ]:
# Importar as bibliotecas
from ipywidgets import interact, FloatSlider, IntSlider
import matplotlib.pyplot as plt
import cv2
import numpy as np

---

#### Carregamento e Exibição da Imagem
Nesta seção, carregamos uma imagem qualquer para análise nos arquivos do google colab. Vamos carregar a imagem primeiro em cores e, em seguida, convertê-la para escala de cinza, pois muitos dos operadores de borda e detecção de cantos funcionam em imagens em tons de cinza.

In [ ]:
# Carregar a imagem em cores
image_path = '/content/tucunaré.webp'  # Substitua pelo caminho da sua imagem
image_color = cv2.imread(image_path)  # Carregar a imagem em cores

# Verificar se a imagem foi carregada corretamente
if image_color is None:
    print("Erro ao carregar a imagem. Verifique o caminho.")
else:
    # Converter de BGR para RGB (OpenCV carrega as cores como BGR)
    image_rgb = cv2.cvtColor(image_color, cv2.COLOR_BGR2RGB)

    # Exibir a imagem em cores
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(image_rgb)
    plt.title("Imagem em Cores (RGB)")
    plt.axis('off')

    # Converter a imagem para escala de cinza
    image_gray = cv2.cvtColor(image_color, cv2.COLOR_BGR2GRAY)

    # Exibir a imagem em escala de cinza
    plt.subplot(1, 2, 2)
    plt.imshow(image_gray, cmap='gray')
    plt.title("Imagem em Escala de Cinza")
    plt.axis('off')

    plt.show()

#### Cálculo dos Gradientes com o Filtro de Sobel
Nesta etapa, aplicamos o filtro de Sobel para calcular os gradientes nas direções 𝑥 e 𝑦. Esses gradientes representam a taxa de variação de intensidade em cada direção, o que é útil para detectar bordas.

Gradiente na Direção X: Calcula mudanças horizontais de intensidade.
Gradiente na Direção Y: Calcula mudanças verticais de intensidade.
Magnitude do Gradiente: Combina os gradientes em ambas as direções para obter uma visão geral das bordas.

In [ ]:
# Calcular o gradiente na direção x usando o filtro de Sobel
sobel_x = cv2.Sobel(image_gray, cv2.CV_64F, 1, 0, ksize=3)

# Calcular o gradiente na direção y usando o filtro de Sobel
sobel_y = cv2.Sobel(image_gray, cv2.CV_64F, 0, 1, ksize=3)

# Exibir os gradientes
plt.figure(figsize=(12, 6))

# Gradiente na direção x
plt.subplot(1, 2, 1)
plt.imshow(sobel_x, cmap='gray')
plt.title("Gradiente na Direção X (Sobel)")
plt.axis('off')

# Gradiente na direção y
plt.subplot(1, 2, 2)
plt.imshow(sobel_y, cmap='gray')
plt.title("Gradiente na Direção Y (Sobel)")
plt.axis('off')

plt.show()

Após calcular a magnitude do gradiente, aplicamos um limiar (threshold) para destacar apenas as regiões onde a intensidade do gradiente é significativa, pois nem todas as variações de intensidade representam bordas importantes na imagem. Algumas mudanças sutis de intensidade podem ser causadas por ruído ou detalhes irrelevantes que não são bordas de interesse.

Configurar um threshold nos permite:

- Filtrar pequenas variações: Ignorar gradientes de baixa intensidade que provavelmente não representam bordas reais.
- Destacar bordas fortes: Preservar apenas as bordas onde a variação de intensidade é suficientemente grande, o que geralmente corresponde a contornos de objetos importantes na imagem.

Assim, o uso de um limiar ajuda a focar na informação relevante, minimizando o ruído e melhorando a clareza da detecção de bordas.

In [ ]:
def update_edges(threshold):
    # Aplicar o limiar para detectar bordas
    _, edges_gradient = cv2.threshold(gradient_magnitude, threshold, 255, cv2.THRESH_BINARY)

    # Exibir a imagem com o gradiente nas direções x, y e a magnitude do gradiente com o limiar
    plt.figure(figsize=(18, 6))

    # Gradiente na direção x
    plt.subplot(1, 3, 1)
    plt.imshow(sobel_x, cmap='gray')
    plt.title("Gradiente na Direção X (Sobel)")
    plt.axis('off')

    # Gradiente na direção y
    plt.subplot(1, 3, 2)
    plt.imshow(sobel_y, cmap='gray')
    plt.title("Gradiente na Direção Y (Sobel)")
    plt.axis('off')

    # Magnitude do gradiente com detecção de bordas
    plt.subplot(1, 3, 3)
    plt.imshow(edges_gradient, cmap='gray')
    plt.title(f"Detecção de Bordas (Threshold={threshold})")
    plt.axis('off')

    plt.show()

# Configurar o slider interativo para ajustar o limiar
interact(update_edges, threshold=IntSlider(min=0, max=255, step=1, value=100))

#### Detector de Bordas de Canny
O detector de bordas de Canny é um dos métodos mais populares e robustos para detecção de bordas em imagens. Ele combina várias etapas, incluindo suavização, cálculo de gradientes, e aplicação de limiares, para identificar bordas de forma precisa. O processo começa com a aplicação de um filtro Gaussiano para suavizar a imagem e reduzir o ruído, o que ajuda a evitar a detecção de bordas falsas. Em seguida, o detector calcula os gradientes da imagem para identificar regiões onde a intensidade muda rapidamente, indicando a presença de uma borda.
<br><br>
O algoritmo Canny utiliza dois parâmetros de limiar (threshold) para controlar a detecção de bordas:

- Limiar Inferior (low_threshold): Define o valor mínimo de intensidade para que um pixel seja considerado parte de uma borda. Pixels com gradientes abaixo desse valor serão ignorados, o que ajuda a reduzir o ruído e a eliminar bordas fracas.
- Limiar Superior (high_threshold): Define o valor de intensidade acima do qual um pixel é definitivamente considerado uma borda forte. Pixels com gradientes maiores que esse valor são considerados bordas reais. Além disso, pixels com gradientes entre o limiar inferior e superior são considerados bordas fracas, mas serão incluídos na borda final se estiverem conectados a bordas fortes.

Essa estratégia de usar dois limiares ajuda a evitar que ruídos ou pequenas variações de intensidade sejam interpretados como bordas, ao mesmo tempo em que preserva a continuidade das bordas reais.

In [ ]:
# Função para aplicar o detector de Canny com limiares ajustáveis
def update_canny(low_threshold, high_threshold):
    # Aplicar o detector de Canny com os limiares ajustados
    edges = cv2.Canny(image_gray, low_threshold, high_threshold)

    # Exibir a imagem resultante
    plt.figure(figsize=(6, 6))
    plt.imshow(edges, cmap='gray')
    plt.title(f"Detector de Bordas de Canny (Limiar Baixo={low_threshold}, Limiar Alto={high_threshold})")
    plt.axis('off')
    plt.show()

# Configurar sliders interativos para os limiares baixo e alto
interact(update_canny,
         low_threshold=IntSlider(min=0, max=255, step=1, value=100, description='Limiar Baixo'),
         high_threshold=IntSlider(min=255, max=500, step=1, value=120, description='Limiar Alto'))

#### Detector de Cantos de Harris
O detector de cantos de Harris é um método utilizado para identificar pontos de interesse (ou "cantos") em uma imagem, que são áreas onde a intensidade muda significativamente em várias direções.

Isso é útil em visão computacional para detectar características estáveis que podem ser rastreadas entre diferentes imagens, como em reconhecimento de objetos ou em reconstrução 3D. O método de Harris baseia-se na análise da matriz de autocorrelação local de gradientes, calculando uma resposta que indica a "força" de um canto em cada pixel.
<br><br>
Os principais parâmetros do detector de Harris são:

- Tamanho do Bloco (block_size): Define o tamanho da vizinhança ao redor de cada pixel onde os gradientes serão analisados. Um valor maior para o tamanho do bloco captura mais informações locais, mas pode suavizar as bordas e cantos menores.
- Tamanho do Kernel de Sobel (ksize): Define o tamanho do kernel usado para calcular os gradientes da imagem nas direções 𝑥 e 𝑦. Normalmente, um kernel de tamanho 3 é suficiente, mas valores maiores podem ser usados para imagens com mais ruído.
- Parâmetro de Sensibilidade (k): Esse é um fator de ajuste que controla a sensibilidade do detector para distinguir entre cantos e bordas. Valores típicos de 𝑘 variam entre 0.04 e 0.06. Valores maiores tornam o detector mais seletivo para cantos, enquanto valores menores aumentam a sensibilidade, podendo detectar também bordas fortes.

Após calcular a resposta de Harris para cada pixel, um limiar é aplicado para destacar apenas os pontos com uma resposta acima de um certo valor, considerando-os como cantos. Esse processo permite que o detector de Harris identifique regiões de interesse que são invariantes a rotação. No entanto, o método não é invariante a mudanças de escala, o que significa que pode ser necessário ajustá-lo para diferentes tamanhos de imagem.


In [ ]:
# Função para aplicar o detector de cantos de Harris com parâmetros ajustáveis
def update_harris(k, threshold_percentage):
    # Calcular a resposta do detector de cantos de Harris
    harris_response = cv2.cornerHarris(image_gray, 3, 3, k)

    # Dilatar a resposta para melhorar a visualização dos cantos
    harris_response_dilated = cv2.dilate(harris_response, None)

    # Definir o limiar para destacar os cantos com base no percentual ajustável
    threshold = threshold_percentage * harris_response_dilated.max()
    corners_image = image_color.copy()
    corners_image[harris_response_dilated > threshold] = [0, 0, 255]  # Destacar cantos em vermelho

    # Exibir a imagem com os cantos destacados
    plt.figure(figsize=(6, 6))
    plt.imshow(cv2.cvtColor(corners_image, cv2.COLOR_BGR2RGB))
    plt.title(f"Cantos Detectados pelo Detector de Harris (k={k}, Limiar={threshold_percentage:.2%})")
    plt.axis('off')
    plt.show()

# Configurar sliders interativos para o parâmetro k e o percentual de limiar
interact(update_harris,
         k=FloatSlider(min=0.04, max=0.1, step=0.01, value=0.06, description='k'),
         threshold_percentage=FloatSlider(min=0.01, max=0.1, step=0.01, value=0.01, description='Limiar %'))


#### Detector de Cantos de Shi-Tomasi
O detector de cantos de Shi-Tomasi, também conhecido como "Good Features to Track" (Boas Características para Rastrear), é uma variante aprimorada do método de Harris. Ele identifica os melhores pontos de interesse (ou "cantos") em uma imagem, especialmente adequado para rastreamento, pois oferece uma detecção mais precisa e estável.

Em vez de calcular uma função de resposta complexa como o detector de Harris, o método de Shi-Tomasi usa os autovalores da matriz de autocorrelação dos gradientes para decidir se um ponto é um canto. Em resumo, um pixel é considerado um canto se o menor dos autovalores locais for maior que um limiar definido.
<br><br>
Os principais parâmetros do detector de Shi-Tomasi são:
- Número Máximo de Cantos (num_corners): Define o número máximo de pontos de interesse a serem detectados. Esse parâmetro permite controlar quantos cantos serão identificados, mantendo apenas os mais fortes até atingir o limite especificado.
- Nível de Qualidade (quality_level): Define o limiar mínimo de qualidade para um ponto ser considerado um canto, expresso como uma fração do autovalor máximo. Por exemplo, um valor de 0.01 significa que apenas cantos com qualidade igual ou superior a 1% do valor máximo serão mantidos. Isso ajuda a garantir que apenas os cantos mais fortes sejam detectados.
- Distância Mínima (min_distance): Especifica a distância mínima entre os cantos detectados, garantindo que eles estejam suficientemente separados. Esse parâmetro é útil para evitar a detecção de múltiplos cantos muito próximos uns dos outros, especialmente em áreas com alta densidade de detalhes.

In [ ]:
# Função para aplicar o detector de Shi-Tomasi com parâmetros ajustáveis
def update_shi_tomasi(num_corners, quality_level, min_distance):
    # Aplicar o detector de Shi-Tomasi
    corners = cv2.goodFeaturesToTrack(image_gray, num_corners, quality_level, min_distance)
    corners = np.intp(corners)  # Converter para inteiros

    # Destacar os cantos na imagem colorida
    shi_tomasi_image = image_color.copy()
    for corner in corners:
        x, y = corner.ravel()
        cv2.circle(shi_tomasi_image, (x, y), 3, (0, 0, 255), -1)  # Cantos em vermelho

    # Exibir a imagem com os cantos detectados
    plt.figure(figsize=(6, 6))
    plt.imshow(cv2.cvtColor(shi_tomasi_image, cv2.COLOR_BGR2RGB))
    plt.title(f"Cantos Detectados com Shi-Tomasi (num_corners={num_corners}, quality_level={quality_level}, min_distance={min_distance})")
    plt.axis('off')
    plt.show()

# Configurar sliders interativos para os parâmetros do Shi-Tomasi
interact(update_shi_tomasi,
         num_corners=IntSlider(min=10, max=1000, step=10, value=100, description='Num Corners'),
         quality_level=FloatSlider(min=0.01, max=0.2, step=0.01, value=0.1, description='Quality Level'),
         min_distance=IntSlider(min=1, max=50, step=1, value=10, description='Min Distance'))